### Romberg Integration

In [1]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import time

from math import erf             # we'll need erf(x) for our test problem


We begin with the trapezoidal rule, a second-order quadrature scheme for closed intervals:
\begin{equation*}
\begin{split}
I &= \sum_{i=0}^{N-2} \frac{1}{2}(f(x_i)+f(x_{i+1})) h \\ 
&= \frac{h (f(x_0)+f(x_{N-1})}{2} + \sum_{i=1}^{N-2} h f(x_i)
\end{split}
\end{equation*}

where the $x_i$ are on a uniform grid with stepsize $h$
$$ x_i = x_0 + i\,h, \quad i=0,\dots,N-1 $$

In [2]:
def trapezoidalRule(func, xmin, xmax, npts):
    assert npts > 1, "trapezoidalRule: npts must be > 1"
    h = (xmax-xmin)/(npts-1)
    In = 0.5*(func(xmin)+func(xmax))
    for j in range(1, npts-1):
        In += func( xmin + j*h )
    return In*h

To test our implementation, let's try using the definition of the error function:
$$ \textrm{erf}(x) = \int_0^x \frac{2}{\sqrt{\pi}}e^{-t^2} dt $$

The integrand is supplied by the function:
(NB: whenever possible, use the Numpy versions of functions so as to allow vectorization...)

In [3]:
def func(x):
    return (2/np.sqrt(np.pi)) * np.exp(-x**2)

Try it out!

In [4]:
n = 8
approx = trapezoidalRule(func,0,1,n)

exact = erf(1)
abserr = exact - approx

print(f'Trapezoidal Rule w/ N={n} points:  {approx:17.14e}')
print(f'                  exact erf(1):  { exact:17.14e}')
print(f'                absolute error:  {abserr:8.3e}')


Trapezoidal Rule w/ N=8 points:  8.41287903150983e-01
                  exact erf(1):  8.42700792949715e-01
                absolute error:  1.413e-03


Let's try timing the execution, using a somewhat larger value for npts

In [5]:
%timeit trapezoidalRule(func,0,1,256)

519 µs ± 6.96 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


We can make this run faster by using numpy to vectorize the computation

In [6]:
def trapezoidalRule(func, xmin, xmax, npts):
    h = (xmax-xmin)/(npts-1)
    return h*(0.5*(func(xmin) + func(xmax)) + func(xmin + np.arange(1,npts-1)*h).sum())

In [7]:
%timeit trapezoidalRule(func,0,1,256)

17.2 µs ± 234 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Let's check that the errors scale as $1/n^2$ -- i.e. that our implementation of the method is second-order accurate as advertised

In [8]:
print(f'Trapezoidal Rule Approximation to erf(1):')
print(f'  npts         approx           error    error*(npts^2)')
for i in range(1,15):
    n = 2**i
    approx = trapezoidalRule(func,0,1,n)
    exact = erf(1)
    abserr = exact - approx
    print(f' {n:5d}  {approx:17.14e}  {abserr:8.3e}  {abserr*(n**2):8.3e}')

Trapezoidal Rule Approximation to erf(1):
  npts         approx           error    error*(npts^2)
     2  7.71743332258054e-01  7.096e-02  2.838e-01
     4  8.34985322634646e-01  7.715e-03  1.234e-01
     8  8.41287903150983e-01  1.413e-03  9.042e-02
    16  8.42393260370657e-01  3.075e-04  7.873e-02
    32  8.42628798170811e-01  7.199e-05  7.372e-02
    64  8.42683361565503e-01  1.743e-05  7.140e-02
   128  8.42696503488131e-01  4.289e-06  7.028e-02
   256  8.42699728980190e-01  1.064e-06  6.973e-02
   512  8.42700527997483e-01  2.650e-07  6.946e-02
  1024  8.42700726841098e-01  6.611e-08  6.932e-02
  2048  8.42700776438705e-01  1.651e-08  6.925e-02
  4096  8.42700788823978e-01  4.126e-09  6.922e-02
  8192  8.42700791918532e-01  1.031e-09  6.920e-02
 16384  8.42700792691951e-01  2.578e-10  6.919e-02


Just to belabour this further, make a plot of the errors

In [9]:
h = []
abserr = []
xmin, xmax = 0,1
exact = erf(1)
for i in range(1,20):
    n = 2**i
    approx = trapezoidalRule(func,0,1,n)
    abserr.append(exact - approx)
    h.append((xmax-xmin)/n)

fig, ax = plt.subplots()
ax.loglog(h, abserr, 'b.', label='points')
ax.set_xlabel("$h$", fontsize=16)
ax.set_ylabel("$|I_n-I|$", fontsize=16)

y = abserr[-1] * (np.asarray(h) / h[-1])**2
ax.loglog(h,y,'y', label=r'$h^2$')
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The errors seem to approach zero with increasing $n$ and they do indeed appear to scale as $h^2$. This is not nearly sufficient testing to be sure of the code's correctness, but for now we will be lazy and accept this as it is.

Now let's see if we can produce a more efficient algorithm (at least for sufficiently
smooth integrands) using Richardson extrapolation.

Assume we have a trapezoidal rule result computed using $2^{(n-1)}$ intervals. The following function returns
the difference between half that result and the result using $2^n$ intervals. In other words, the following code gives the result of adding an additional quadrature point at the center of each of the $2^{(n-1)}$ intervals.
In yet other words, if $I_n$ is the trapezoidal rule result for $2^n$ intervals, then $$ I_n = \frac{1}{2}I_{n-1} + \textrm{nextTrapezoidalRule(f,xmin,xmax,n)} $$

In [10]:
def nextTrapezoidalRule(func, xmin, xmax, n):
    """
    Return the difference between the trapezoidal rule approximation
    for 2^n intervals and one-half the trapezoidal rule approximation
    for 2^(n-1) intervals.
    """
    npts = 2**n
    h = (xmax-xmin)/npts
    In = 0
    for j in range(1, npts//2+1):
        In += func( xmin + (2*j-1)*h )
    return h*In

We can time the execution of this code as follows:

In [11]:
n = 19
%timeit nextTrapezoidalRule(func, 0, 1, n)

574 ms ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


This seems rather a slow implementation. Most of the work is of course in the `for` loop. Let's try to vectorize by using numpy, creating an array of results to which we can apply numpy.sum:

In [12]:
def nextTrapezoidalRule(func, xmin, xmax, n):
    """
    Return the difference between the trapezoidal rule approximation
    for 2^n intervals and one-half the trapezoidal rule approximation
    for 2^(n-1) intervals.
    """
    npts = 2**n
    h = (xmax-xmin)/npts
    return h * ( func( xmin + (2*np.arange(1,npts//2+1)-1)*h) ).sum()

We can now time this implementation

In [13]:
n = 19
%timeit nextTrapezoidalRule(func, 0, 1, n)

2.51 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Let's check that the Numpy implementation behaves as expected.

In [14]:
xmin = 0
xmax = 1

print("Trapezoidal rule:")
print(f'  npts         approx           error       error*(npts^2)')
I = 0.5*(xmax-xmin)*(func(xmin)+func(xmax))
print(f"{2:8d}  {I:17.15f}")
npts = 2
for n in range(1,22):
    I = 0.5*I + nextTrapezoidalRule(func, 0, 1, n)
    npts *= 2
    abserr = I-erf(1)
    print(f"{npts:8d}  {I:17.15f}  {abserr: e}  {abserr*(npts**2): e}")

Trapezoidal rule:
  npts         approx           error       error*(npts^2)
       2  0.771743332258054
       4  0.825262955596749  -1.743784e-02  -2.790054e-01
       8  0.838367777441205  -4.333016e-03  -2.773130e-01
      16  0.841619221244768  -1.081572e-03  -2.768824e-01
      32  0.842430505490233  -2.702875e-04  -2.767744e-01
      64  0.842633227681257  -6.756527e-05  -2.767473e-01
     128  0.842683902044949  -1.689090e-05  -2.767406e-01
     256  0.842696570249296  -4.222700e-06  -2.767389e-01
     512  0.842699737276221  -1.055673e-06  -2.767385e-01
    1024  0.842700529031442  -2.639183e-07  -2.767384e-01
    2048  0.842700726970153  -6.597956e-08  -2.767383e-01
    4096  0.842700776454825  -1.649489e-08  -2.767383e-01
    8192  0.842700788825992  -4.123722e-09  -2.767383e-01
   16384  0.842700791918784  -1.030931e-09  -2.767383e-01
   32768  0.842700792691982  -2.577326e-10  -2.767383e-01
   65536  0.842700792885282  -6.443313e-11  -2.767382e-01
  131072  0.8427007929336

and make the same plot as before:

In [15]:
h = []
abserr = []
xmin, xmax = 0,1
exact = erf(1)
I = 0.5*(xmax-xmin)*(func(xmin)+func(xmax))
for i in range(1,20):
    n = 2**i
    I = 0.5*I + nextTrapezoidalRule(func, 0, 1, i)
    abserr.append(exact - I)
    h.append((xmax-xmin)/n)

fig, ax = plt.subplots()
ax.loglog(h, abserr, 'b.')
ax.set_xlabel("$h$", fontsize=16)
ax.set_ylabel("$|I_n-I|$", fontsize=16)

y = abserr[-1] * (np.asarray(h) / h[-1])**2
ax.loglog(h,y,'y');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We can now use this more-efficient version of the trapezoidal rule to
implement Romberg integration (for a derivation of which, see the class notes).

This version prints out the tableau of intermediate results:

In [16]:
def rombergIntegration(func, xmin, xmax, epsrel, nmax=10):
    """
   Use Richardson extrapolation to compute the integral of f(x)
   from xmin to xmax with a relative error less than epsrel,
   using trapezoidal rules with up to 2^nmax intervals.
   """

    I = np.zeros((nmax+1,nmax+1)) # save space for the tableau

    I[0,0] = 0.5*(xmax-xmin)*(func(xmin)+func(xmax))  # initial trapezoidal rule
    print(f"{0:3d}  {I[0,0]: 17.15f}")

    for n in range(1,nmax+1):
        I[n,0] = 0.5 * I[n-1,0] + nextTrapezoidalRule(func, xmin, xmax, n)

        print(f"{n:3d}  {I[n,0]: 17.15f}", end=" ")
        for k in range(1, n+1):
            q = 4**k
            I[n,k] = (q*I[n,k-1] - I[n-1,k-1])/(q-1)
            print(f"{I[n,k]: 17.15f}", end=" ")
            if abs(I[n,k]-I[n,k-1]) <= epsrel*abs(I[n,n-1]):
                print()
                return(I[n,k])
        print()


    print(f"romberg failed to converge in {nmax:d} rows")

In [18]:
nmax = 12
eps = 1e-12
n = 5
approx = rombergIntegration(func, xmin, xmax, eps, n)
exact = erf(1)
abserr = approx-exact
relerr = abserr/exact
print(f'\n      {approx:17.15f}   relerr = {relerr:8.3e}   abserr = {abserr:8.3e}')
print(f'      {erf(1):17.15f}')

  0   0.771743332258054
  1   0.825262955596749  0.843102830042981 
  2   0.838367777441205  0.842736051389357  0.842711599479115 
  3   0.841619221244768  0.842703035845956  0.842700834809729  0.842700663941961 
  4   0.842430505490233  0.842700933572054  0.842700793420461  0.842700792763488  0.842700793268671 
  5   0.842633227681257  0.842700801744932  0.842700792956457  0.842700792949092  0.842700792949820 

      0.842700792949820   relerr = 1.245e-13   abserr = 1.049e-13
      0.842700792949715


Romberg integration achieved a relative error of 1e-13 using $2^5 = 32$ function evaluations. Referring to the results above, the trapezoidal rule alone required $2^{20}\approx 10^6$ function evaluations to achieve the same relative accuracy!

Removing the print statements, we have

In [19]:
def rombergIntegration(func, xmin, xmax, epsrel, nmax=10):
    """
   Use Richardson extrapolation to compute the integral of f(x)
   from xmin to xmax with a relative error less than epsrel,
   using trapezoidal rules with up to 2^nmax intervals.
   """

    I = np.zeros((nmax+1,nmax+1)) # save space for the tableau

    I[0,0] = 0.5*(xmax-xmin)*(func(xmin)+func(xmax))  # initial trapezoidal rule
    neval =2
    
    for n in range(1,nmax+1):
        I[n,0] = 0.5 * I[n-1,0] + nextTrapezoidalRule(func, xmin, xmax, n)
        neval += 2**(n-1)
        
        for k in range(1, n+1):
            q = 4**k
            I[n,k] = (q*I[n,k-1] - I[n-1,k-1])/(q-1)

            if abs(I[n,k]-I[n,k-1]) <= epsrel*abs(I[n,n-1]):
                return(I[n,k]), neval

    print(f"romberg failed to converge in {nmax:d} rows")

In [21]:
nmax = 12
eps = 1e-10
n = 5
tr = %timeit -o rombergIntegration(func, xmin, xmax, eps, n)
approx, neval = rombergIntegration(func, xmin, xmax, eps, n)
exact = erf(1)
abserr = approx-exact
relerr = abserr/exact
print(f'\n      {approx:17.15f}   relerr = {relerr:8.3e}   abserr = {abserr:8.3e}')
print(f'      {erf(1):17.15f}')
print(f'neval: {neval}')

103 µs ± 918 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

      0.842700792949820   relerr = 1.245e-13   abserr = 1.049e-13
      0.842700792949715
neval: 33


In [22]:
n = 2**19 + 290000 # adjust to get error the same as for romberg
tt = %timeit -o trapezoidalRule(func, xmin, xmax, n)
approx = trapezoidalRule(func, xmin, xmax, n)
exact = erf(1)
abserr = approx-exact
relerr = abserr/exact
print(f'\n      {approx:17.15f}   relerr = {relerr:8.3e}   abserr = {abserr:8.3e}')
print(f'      {erf(1):17.15f}')
print(f'neval: {n}')
print()
print(f"romberg speedup: {tt.average/tr.average}")

11.2 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

      0.842700792949611   relerr = -1.232e-13   abserr = -1.038e-13
      0.842700792949715
neval: 814288

romberg speedup: 108.37465977801912


***

### Gaussian Quadrature

The trapezoidal rule and related methods, the abscissae ($x$-values) are chosen to be on a uniform mesh and
various weights $w_i$ are used in formulae like
$$ \int_a^b f(x) dx \approx \sum_{i=0}^{N-1} w_i f(x_i) $$
The trapezoidal rule's weights, for example, are
$$ w_i = \begin{cases} \frac{1}{2}, & i=0, N-1\\ 1, & else \end{cases} $$
The methods are of fixed order, and the error is a power law in the stepsize; again for the trapezoidal rule, $h^2$.

In Gaussian quadrature one exploits twice the degrees of freedom by choosing the $\left\{x_i\right\}$ to achieve greater accuracy from the same number of points. The method works for integrals of the form
$$ \int_a^b W(x) f(x) dx \approx \sum _{i=0}^{N-1} w_i f(x_i) $$
where $W$ is some "weighting function" and is exact when $f(x)$ is a polynomial of degree $2N-1$ or less. An interesting feature is that the error decreases exponentially with increasing $N$ because the *order* of the method, and not just the density of points, increases with $N$.

Scipy provides weights and abscissae for some common weighting functions $W(x)$:
\begin{equation*}
\begin{array}{llll}
W(x) &= 1, & -1< x<1 & \textrm{Gauss-Legendre} & \textrm{scipy.special.roots_legendre}\\
W(x) &= (1-x^2)^{1/2}, & -1< x<1 &\textrm{Gauss-Chebyshev U} & \textrm{scipy.special.roots_chebyu}\\
W(x) &= (1-x^2)^{-1/2}, & -1< x<1 &\textrm{Gauss-Chebyshev T} & \textrm{scipy.special.roots_chebyt}\\
W(x) &= x^\alpha e^{-x} & 0<x<\infty & \textrm{Gauss-Laguerre} & \textrm{scipy.special.roots_genlaguerre}\\
W(x) &= e^{-x^2} & -\infty<x<\infty & \textrm{Gauss-Hermite} & \textrm{scipy.special.roots_hermite}\\
W(x) &= (1-x)^\alpha (1+x)^\beta  & -1< x<1 & \textrm{Gauss-Jacobi} & \textrm{scipy.special.roots_jacobi}
\end{array}
\end{equation*}
among others.

The theory of how to find these weights and abscissae are beyond the scope of this course, but they are related to generating sets of polynomials orthogonal under the inner product
$$ \langle f|g\rangle = \int_a^b W(x) f(x) g(x) dx $$
as one might guess from the set of functions provided by Scipy. The abscissae are the roots of these polynomials. If you need a different, non-standard weighting function, see e.g. *Numerical Recipes* and references therein.

As an example, let's compute the integral of some function $f(x)$ over a normal distribution
$$  \frac{1}{\sigma\sqrt{\pi}} \int_{-\infty}^\infty \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right) f(x) dx $$
With the change of variable 
$$ y = \sqrt{2}\sigma x + \mu $$
this is
$$ \frac{1}{\sqrt{\pi}} \int_{-\infty}^\infty e^{-y^2} f(\sqrt{2}\sigma y + \mu) dy $$
This has a Gauss-Hermite weighting function so we will use `scipy.special.roots_hermite` to obtain the abscissae and weights.

We'll choose $f(x) = x^2 \cos^2(x)$; not a polynomial, but a smooth function in the sense that it is everywhere close to a polynomial.

In [23]:
from scipy import special
def f(x):
    sigma = 1
    mu = 4
    y = np.sqrt(2)*sigma*x+mu
    return y**2*np.cos(y)**2

print(f"N    Integral")
for N in range(2,20):
    xh, wh = special.roots_hermite(N)
    ans = np.sum( wh * f(xh) )
    print(f"{N:2d}  {ans:.15f}")

N    Integral
 2  9.599939889313887
 3  15.748543016370952
 4  11.700629401823951
 5  13.316164765932156
 6  12.853329693620326
 7  12.956780433196581
 8  12.937822856801386
 9  12.940766552187700
10  12.940370016542678
11  12.940417181088277
12  12.940412159302927
13  12.940412643236256
14  12.940412600646148
15  12.940412604095117
16  12.940412603836489
17  12.940412603854536
18  12.940412603853360
19  12.940412603853433


Let's try Romberg integration as a check. Because the integrand goes rapidly to zero with increasing $|x|$,
we can choose finite limits: $[-7,7]$ should do just fine:

In [24]:
x = np.array([-7,7])
f(x)*np.exp(-x**2)

array([1.56902809e-20, 5.61476467e-21])

In [25]:
ans, neval = rombergIntegration(lambda x: f(x)*np.exp(-x**2), -7, 7, 1e-14)
print(f"{neval}   {ans:17.15f}")

129   12.940412603853432


In [26]:
fig, ax = plt.subplots()
x = np.linspace(-5,5,1000)
y = f(x)*np.exp(-x**2)
ax.plot(x,y,label=r"$x^2 cos^2(x) G(x,\sigma,\mu$")
ax.plot(xh,wh*10,'r.',label=r"$x_i, 10w_i$")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This idea can be extended to multidimensional integrals, though with some difficulty. One interesting example is the *spherical t-design*. This is a set of $n>t$ points on the surface of a sphere such that the average value of a polynomial of degree $t$ over the sphere is the average value of the polynomial at the points. This average value is, of course, the integral of the polynomial over the surface of the sphere.